In [ ]:
import json

with open("data/llm-papers.json") as f:
    pdf_infos = json.load(f)

pdf_urls = [pdf["url"] for pdf in pdf_infos]

In [ ]:
from langchain.document_loaders import PyPDFLoader


def load_and_split_pdf_url(pdf_url):
    loader = PyPDFLoader(pdf_url)
    doc = loader.load_and_split()
    return doc


docs = []

for i in range(50):
    loader = PyPDFLoader(pdf_urls[i])
    doc = loader.load_and_split()
    docs.append(doc)


# sample = load_and_split_pdf_url(pdf_urls[0])

In [ ]:
docs = sum(docs, [])

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=700)

sample_split = text_splitter.split_documents(docs)

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

embeddingslist = []


def embed_single_document(doc: str) -> list:
    text = doc.page_content
    embeddings = model.encode(text)

    return embeddings


for i in range(5):
    embeddingslist.append(embed_single_document(docs[i]))

embeddingslist

In [ ]:
upsert = []

for text, embedding in zip(docs[0:5], embeddingslist):
    upsert.append((text, embedding))
    break

In [ ]:
upsert[0]

In [ ]:
def create_vectorstore_of_text(docs):
    from langchain.embeddings import FakeEmbeddings
    from langchain.vectorstores.faiss import FAISS
    from langchain.text_splitter import RecursiveCharacterTextSplitter

    text_splitter = RecursiveCharacterTextSplitter()

    docs = text_splitter.split_documents(docs)

    # FAISS.from_documents(List[Document], List[List[float]])
    faiss_index = FAISS.from_documents(docs, embedding=FakeEmbeddings(size=200))

    faiss_index.save_local("vectorstore/llm-papers-faiss")

    return faiss_index


create_vectorstore_of_text(docs=sample_split)